In [1]:
import numpy as np
import cupy as cp
import warnings
import os

gpu_available = True

from few.trajectory.inspiral import EMRIInspiral
from few.amplitude.romannet import RomanAmplitude
from few.amplitude.interp2dcubicspline import Interp2DAmplitude
from few.waveform import FastSchwarzschildEccentricFlux, SlowSchwarzschildEccentricFlux, GenerateEMRIWaveform
from few.utils.utility import get_overlap, get_mismatch, get_separatrix, get_fundamental_frequencies
from few.utils.ylm import GetYlms
from few.utils.modeselector import ModeSelector
from few.summation.interpolatedmodesum import CubicSplineInterpolant
from few.utils.constants import *
from few.utils.baseclasses import SchwarzschildEccentric, Pn5AAK, ParallelModuleBase
from few.waveform import AAKWaveformBase, Pn5AAKWaveform
from few.summation.aakwave import AAKSummation

import matplotlib.pyplot as plt

from few.utils.utility import get_p_at_t

import time
import warnings

use_gpu=True    

In [2]:
from fisher import StableEMRIFisher

In [3]:
#set initial parameters
M = 1e6
mu = 10
a = 0.9
p0 = 8.05
e0 = 0.1
iota0 = 0.0 #equatorial
Y0 = np.cos(iota0)
Phi_phi0 = 2
Phi_theta0 = 3
Phi_r0 = 1.5

qS = 0.2
phiS = 0.2
qK = 0.8
phiK = 0.8
dist = 1
mich = True #mich = True implies output in hI, hII long wavelength approximation
dt = 10.0
T = 0.1

inspiral_kwargs = {
        "DENSE_STEPPING": 0,
        "max_init_len": int(1e3),
        "err": 1e-10,  # To be set within the class
        "use_rk4": True,
        }
# keyword arguments for summation generator (AAKSummation)
sum_kwargs = {
    "use_gpu": True,  # GPU is available for this type of summation
    "pad_output": True,
}
amplitude_kwargs = {
    "specific_spins":[0.80, 0.9, 0.95]
    }

Waveform_model = GenerateEMRIWaveform('KerrEccentricEquatorialFlux', inspiral_kwargs=inspiral_kwargs, sum_kwargs=sum_kwargs, use_gpu=use_gpu)

In [ ]:
waveform = cp.asnumpy(Waveform_model(M, mu, a, p0, e0, Y0, 
                          dist, qS, phiS, qK, phiK, 
                          Phi_phi0, Phi_theta0, Phi_r0, 
                          mich=mich, dt=dt, T=T))

plt.plot(waveform[-1000:])

In [ ]:
#varied parameters
param_names = ['M','mu','a','p0','e0']

#initialization
sef = StableEMRIFisher(M, mu, a, p0, e0, Y0, dist, qS, phiS, qK, phiK,
              Phi_phi0, Phi_theta0, Phi_r0, dt, T, EMRI_waveform_gen=Waveform_model,
              param_names=param_names, stats_for_nerds=False,
              filename='TestRun', CovMat=True, CovEllipse=True)

#execution
sef()